In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob, os

In [2]:
#path = r'C:\Users\user\Desktop\BoT-Iot Dataset\Entire Dataset'
path = r'/mnt/data/dino/dataset/'


csvs = "AWID-CLS-R-Trn/1"
csv_paths = os.path.join(path, csvs)
csv_glob = glob.glob(csv_paths)

header_path = os.path.join(path, 'header.csv')
header = pd.read_csv(header_path, header=None)

awid_arr = []
for csv in csv_glob:
    awid_arr.append(
        pd.read_csv(csv, header=None, low_memory=False,
                    names=header.iloc[0]
                   )        
     )
awid = pd.concat(awid_arr)

In [3]:
awid

,frame.interface_id,frame.dlt,frame.offset_shift,frame.time_epoch,frame.time_delta,frame.time_delta_displayed,frame.time_relative,frame.len,frame.cap_len,frame.marked,...,wlan.qos.priority,wlan.qos.eosp,wlan.qos.ack,wlan.qos.amsdupresent,wlan.qos.buf_state_indicated,wlan.qos.bit4,wlan.qos.txop_dur_req,wlan.qos.buf_state_indicated2,data.len,class
0,0,?,0.0,1.393661e+09,0.000000,0.000000,0.000000,261,261,0,...,?,?,?,?,?,?,?,?,?,normal
1,0,?,0.0,1.393661e+09,0.024271,0.024271,0.024271,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
2,0,?,0.0,1.393661e+09,0.001631,0.001631,0.025902,185,185,0,...,?,?,?,?,?,?,?,?,?,normal
3,0,?,0.0,1.393661e+09,0.055325,0.055325,0.081227,159,159,0,...,?,?,?,?,?,?,?,?,?,normal
4,0,?,0.0,1.393661e+09,0.000415,0.000415,0.081642,54,54,0,...,?,?,?,?,?,?,?,?,?,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795570,0,?,0.0,1.393665e+09,0.000006,0.000006,3599.522705,40,40,0,...,?,?,?,?,?,?,?,?,?,normal
1795571,0,?,0.0,1.393665e+09,0.005067,0.005067,3599.527772,148,148,0,...,?,?,?,?,?,?,?,?,?,normal
1795572,0,?,0.0,1.393665e+09,0.010242,0.010242,3599.538014,54,54,0,...,?,?,?,?,?,?,?,?,?,normal
1795573,0,?,0.0,1.393665e+09,0.000007,0.000007,3599.538021,40,40,0,...,?,?,?,?,?,?,?,?,?,normal


In [4]:
awid["class"].value_counts()

normal           1633190
injection          65379
impersonation      48522
flooding           48484
Name: class, dtype: int64

# Attack Dictionary

In [5]:
attack_sam = 1000
benign_sam = 10000

awid_categ = {
           # 'flooding' : pd.concat([
            #             awid[awid['class'] == 'flooding'].sample(n=attack_sam),
             #            awid[awid['class'] == 'normal'].sample(n=benign_sam),
              #       ]),
              'impersonation'  : pd.concat([
                          awid[awid['class'] == 'impersonation'][:attack_sam],
                          awid[awid['class'] == 'normal'][:benign_sam],
                      ]),
           # 'injection'  : pd.concat([
            #             awid[awid['class'] == 'injection'].[:attack_sam],
             #            awid[awid['class'] == 'normal'].[:benign_sam],
              #       ])
}

# Preprocessing

In [6]:
for cat in awid_categ:
    awid_categ[cat]=awid_categ[cat].replace('?',np.nan)
    awid_categ[cat]=awid_categ[cat].dropna(axis=1)
    awid_categ[cat].dropna(axis=1, inplace=True)
    
   # awid_categ[cat].loc[ awid_categ[cat]['class']=='flooding','class' ]=1
    awid_categ[cat].loc[ awid_categ[cat]['class']=='impersonation','class' ]=1
   # awid_categ[cat]_df.loc[ awid_categ[cat]_df['class']=='injection','class' ]=1
    awid_categ[cat].loc[ awid_categ[cat]['class']=='normal','class' ]=0
    awid_categ[cat]['class'] = awid_categ[cat]['class'].astype(int)
    
    for f in ["radiotap.present.reserved", "wlan.fc.type_subtype", "wlan.fc.ds"]:
        awid_categ[cat][f] = awid_categ[cat][f].apply(int, base=16)

    

# Modeling & Feature Drop

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score



In [8]:
CLF = {
        'KNN': [ KNeighborsClassifier(), 420,9000],
        'RFR': [ RandomForestClassifier(random_state=42), 150,9000],
        'LRE': [ LogisticRegression(), 992,9000],
}
def model( df ):

    for cl in CLF:
        print('ClASSIFIER :',cl)
        clf = CLF[cl][0]
        nattack = CLF[cl][1]
        nbenign = CLF[cl][2]

        df_train_attack = df[df['class'] == 1][:nattack]
        df_train_benign = df[df['class'] == 0][:nbenign]
        df_train = pd.concat([df_train_attack, df_train_benign])

        df_test_attack = df[df['class'] == 1][nattack+1:]
        df_test_benign = df[df['class'] == 0][nbenign+1:]
        df_test = pd.concat([df_test_attack, df_test_benign])

        X_train = df_train.drop( ["class"], axis=1)
        X_test = df_test.drop( ["class"], axis=1)
        y_train = df_train['class']
        y_test = df_test['class']

        clf.fit(X_train, y_train)
        print('score = ',clf.score(X_test, y_test))
        #y_preds = clf.predict(X_test)
        #print( classification_report(y_test, y_preds) )
        #print( confusion_matrix(y_test, y_preds))
        #print('')
        #print('========')
    print('')
    print('=============================')
    print('')

In [ ]:
for cat in awid_categ:
    print('============',cat,'============')
    df = awid_categ[cat]
    model(df)
    headers = list(df.columns)
    headers.remove('class')
    #print(list(headers))
    for h in headers:
        print('===> DROPPING :',h)
        #df_drop = df.drop([h], axis=1) #Independent
        #model(df_drop)  # Independent
        df.drop([h], axis=1, inplace=True) #group selection
        model(df) # group

============ impersonation ============
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score =  0.9930417495029821


===> DROPPING : frame.interface_id
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score =  0.9930417495029821


===> DROPPING : frame.offset_shift
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score =  0.9930417495029821


===> DROPPING : frame.time_epoch
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score =  1.0


===> DROPPING : frame.time_delta
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score =  1.0


===> DROPPING : frame.time_delta_displayed
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score =  1.0


===> DROPPING : frame.time_relative
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score = 

/home/dino/.conda/envs/dino-ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


 1.0


===> DROPPING : frame.len
ClASSIFIER : KNN
score =  1.0
ClASSIFIER : RFR
score =  1.0
ClASSIFIER : LRE
score =  1.0


===> DROPPING : frame.cap_len
ClASSIFIER : KNN


/home/dino/.conda/envs/dino-ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


score =  0.991128010139417
ClASSIFIER : RFR
score =  0.9897186147186147
ClASSIFIER : LRE
score =  1.0


...

score =  0.991128010139417
ClASSIFIER : RFR
score =  0.9902597402597403
ClASSIFIER : LRE
score =  1.0


===> DROPPING : radiotap.present.dbm_tx_power
ClASSIFIER : KNN


/home/dino/.conda/envs/dino-ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


score =  0.991128010139417
ClASSIFIER : RFR
score =  0.9902597402597403
ClASSIFIER : LRE
score =  1.0


===> DROPPING : radiotap.present.antenna
ClASSIFIER : KNN


/home/dino/.conda/envs/dino-ml/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


score =  0.991128010139417
ClASSIFIER : RFR
score =  0.9897186147186147
ClASSIFIER : LRE
score =  1.0


===> DROPPING : radiotap.present.db_antsignal
ClASSIFIER : KNN
score =  0.991128010139417
ClASSIFIER : RFR
score =  0.9897186147186147
ClASSIFIER : LRE
score =  1.0


===> DROPPING : radiotap.present.db_antnoise
ClASSIFIER : KNN
score =  0.991128010139417
ClASSIFIER : RFR
score =  0.9897186147186147
ClASSIFIER : LRE
score =  1.0


===> DROPPING : radiotap.present.rxflags
ClASSIFIER : KNN
score =  0.991128010139417
ClASSIFIER : RFR
score =  0.9902597402597403
ClASSIFIER : LRE
score =  1.0


===> DROPPING : radiotap.present.xchannel
ClASSIFIER : KNN
